In [37]:
from tabula import read_pdf
from tabulate import tabulate
import pandas as pd
import numpy as np

import openpyxl


# Using Tabula for Table Extraction

In [ ]:
df = read_pdf('ESG-Frameworks/Mapping-Standards/sdg-gri.pdf', stream=True, pages = 3, area = [80.51, 90.42, 561.96, 814.18], multiple_tables=False )


In [ ]:
for i in range(4, 73):
    df[0] = df[0].append(read_pdf('ESG-Frameworks/Mapping-Standards/sdg-gri.pdf', stream=True, pages = i, area = [80.51, 90.42, 561.96, 814.18], multiple_tables=False )[0], ignore_index=True)
    

In [ ]:
df[0].rename(columns = {'Sources':'Source'}, inplace = True)

In [ ]:
for i in range(74, 99):
    df[0] = df[0].append(read_pdf('ESG-Frameworks/Mapping-Standards/sdg-gri.pdf', stream=True, pages = i, area = [80.51, 90.42, 561.96, 814.18], multiple_tables=False )[0], ignore_index=True)
    

In [ ]:
df[0].to_csv('ESG-Frameworks/Mapping-Standards/SDG-GRI-DF.csv')

# Finding links between Frameworks SDG AND GRI

In [24]:
df = pd.read_csv('ESG-Frameworks/Mapping-Standards/SDG-GRI-DF.csv')
# df.head(3)

In [25]:
df = df.drop(['Unnamed: 0'], axis=1)

In [26]:
df = df.replace(np.nan, '', regex=True)

# Solving Disclousure Column extraction Issue 

In [30]:
df['Target_'] = df['Target']

In [31]:
df = df[["Target", "Target_", "Available Business Disclosures", "Disclosure"]]
# df

In [32]:
df = df.drop(labels = 'Target_',axis = 1).groupby(df['Target_'].mask(df['Target_']=='').ffill()).agg('\n'.join).reset_index()

In [34]:
df.rename(columns = {'Target_':'Target'}, inplace = True)

In [36]:
df.to_csv('ESG-Frameworks/Mapping-Standards/SDG-GRI.csv')


# Mapping Collected Datafream on given excel file

In [64]:
output_path = 'ESG-Frameworks/Mapping-Standards/SDG-GRI-DF.csv'
excelFile = 'ESG-Frameworks/Outputs/testing.xlsx'

def Mapping(file, df):
    wb_obj = openpyxl.load_workbook(file)
    
    # sheet_obj = wb_obj.active
    
    # cell_obj = sheet_obj.cell(row = 5, column = 1)
    
    print(wb_obj.sheetnames)
Mapping(excelFile, output_path)

["SDG's", 'GRI 2016', 'TCFD', 'COH4B']


NameError: name 'cell_obj' is not defined